In [2]:
import numpy as np

Actions_1 = ['LEFT','RIGHT','UP','DOWN']
def getBoundaryConditionCheck(value,s):
  if (value > 4) or (value < 0) :
   return -1

def returnNeighbourState(s,a,state_space,V,p):
    row,col = state_space[s]['row_col']
    if a == "LEFT":
       left = col-1
       check = getBoundaryConditionCheck(left,s)
       if check == -1 :
          return (float(p),s,V[s])
       return (float(p),s-1,V[s-1])

    if a == "RIGHT":
       right = col+1
       check = getBoundaryConditionCheck(right,s)
       if check == -1 :
          return (float(p),s,V[s])
       return (float(p),s+1,V[s+1])
    if a == "UP":
       top = row-1
       check = getBoundaryConditionCheck(top,s)
       if check == -1 :
          return (float(p),s,V[s])
       return (float(p),s-5,V[s-5])
    if a == "DOWN":
       down = row+1
       check = getBoundaryConditionCheck(down,s)
       if check == -1 :
          return (float(p),s,V[s])
       return (float(p),s+5,V[s+5])


def getAssociatedNeighbourStates(V, s, a, state_space) :
    Actions_1 = ['LEFT','RIGHT','UP','DOWN']
    Actions_1.remove(a)
    neighbour_state_List = list()
    neighbour_state = returnNeighbourState(s,a,state_space, V, float(0.8))
    neighbour_state_List.append(neighbour_state)
    for i in Actions_1:
     neighbour_state = returnNeighbourState(s,i,state_space,V, float(0.04))
     neighbour_state_List.append(neighbour_state)
    return neighbour_state_List

def getReward(next_s, state_space):
    return V[next_s]


def eval_state_action(V, s, a, state_Space, gamma=1.0):  # for each environment, experiment with 3 different values, say 0.90, 0.5 and 0.1

    for p, next_s, rew in getAssociatedNeighbourStates(V, s, a, state_space) :
      x = state_Space[next_s]
      sx = np.sum(-1 + p*(gamma * state_Space[next_s]['reward']))
      return np.sum(sx)

def value_iteration(state_space, nA, V):
    it = 0
    nS = len(V)
    eps=0.1
    while True:
        delta = 0
        # update the value of each state and check for convergence
        for s in range(nS):
            old_v = V[s]
            V[s] = np.max([eval_state_action(V, s, a, state_space) for a in nA]) #Get the Bellman updated value for all actions and choose the action which gives the maximum value
            delta = max(delta, np.abs(old_v - V[s]))
        if delta < eps:
            break
        else:
            print('Iter:', it, ' delta:', np.round(delta, 5))
    return V

def run_episodes(V, Actions, tax_loc, p1):
    '''
    Run some test games
    '''
    nA = ['LEFT','RIGHT','UP','DOWN']
    tot_rew = 0
    for _ in range(100):
        done = 0
        path = list()
        while tax_loc != p1:
            action = np.argmax([eval_state_action(V, tax_loc, a, state_space) for a in nA])
            action = Actions[action]
            _ ,next_state, reward = returnNeighbourState(tax_loc, action, state_space, V,0)
            path.append(next_state)
            tax_loc = next_state
            tot_rew += reward
            if tax_loc == p1:
                break
            return path

rows, cols = (5, 5)
Value_State = [0 for i in range(cols*rows)]

##Create the enviroment
def createSP() :
  m=0;
  row_col = tuple()
  reward = 0
  State_Space = dict()
  for i in range(cols):
    for j in range(rows) :
      State_Space[m] = {'row_col':(i,j),'reward':0}
      m+=1
  return State_Space

##Create the state_Space
rows, cols = (5, 5)
state_space = createSP()
Actions = ['LEFT','RIGHT','UP','DOWN','PICKUP','DROP']

L1 = 0 ##Location 1
L2 = 14  ##Location 2
L3 = 24  ##Location 3
TaxiLocation = 6 ##TaxiLocation
Obstacle = 12 ##Obstacle

## Passenger P1 pickup
state_space[L1]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,TaxiLocation,L1)
V = np.array(V)
print(f'\n Grid for TaxiLocation to L1 : \n {V.reshape((5,5))}')

## Passenger P1 Drop
state_space[L2]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,L1,L2)
V = np.array(V)
print(f'\n Grid for TaxiLocation to L4 P1 drop: \n {V.reshape((5,5))}')

##Passenger P2 pickup
state_space[L1]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,L1,L2)
V = np.array(V)
print(f'\n Grid for TaxiLocation to P2 at L2 : \n {V.reshape((5,5))}')


##Passenger P2 drop
state_space[L2]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,L2,L1)
V = np.array(V)
print(f'\n Grid for TaxiLocation to P2 drop at L1: \n{V.reshape((5,5))}')

print(f'\n For other 2 episode\n')

## Passenger P1 pickup
state_space[L1]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,TaxiLocation,L1)
V = np.array(V)
print(f'\n Grid for TaxiLocation to L1 : \n {V.reshape((5,5))}')

## Passenger P1 Drop
state_space[L2]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,L1,L2)
V = np.array(V)
print(f'\n Grid for TaxiLocation to L4 P1 drop: \n {V.reshape((5,5))}')

##Passenger P2 pickup
state_space[L1]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,L1,L2)
V = np.array(V)
print(f'\n Grid for TaxiLocation to P2 at L2 : \n {V.reshape((5,5))}')


##Passenger P2 drop
state_space[L2]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,L2,L1)
V = np.array(V)
print(f'\n Grid for TaxiLocation to P2 drop at L1: \n{V.reshape((5,5))}')


print(f'\n For  3rd episode\n')

## Passenger P1 pickup
state_space[L1]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,TaxiLocation,L1)
V = np.array(V)
print(f'\n Grid for TaxiLocation to L1 : \n {V.reshape((5,5))}')

## Passenger P1 Drop
state_space[L2]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,L1,L2)
V = np.array(V)
print(f'\n Grid for TaxiLocation to L4 P1 drop: \n {V.reshape((5,5))}')

##Passenger P2 pickup
state_space[L1]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,L1,L2)
V = np.array(V)
print(f'\n Grid for TaxiLocation to P2 at L2 : \n {V.reshape((5,5))}')


##Passenger P2 drop
state_space[L2]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,L2,L1)
V = np.array(V)
print(f'\n Grid for TaxiLocation to P2 drop at L1: \n{V.reshape((5,5))}')

print(f'\n For  4th episode\n')

## Passenger P1 pickup
state_space[L1]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,TaxiLocation,L1)
V = np.array(V)
print(f'\n Grid for TaxiLocation to L1 : \n {V.reshape((5,5))}')

## Passenger P1 Drop
state_space[L2]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,L1,L2)
V = np.array(V)
print(f'\n Grid for TaxiLocation to L4 P1 drop: \n {V.reshape((5,5))}')

##Passenger P2 pickup
state_space[L1]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,L1,L2)
V = np.array(V)
print(f'\n Grid for TaxiLocation to P2 at L2 : \n {V.reshape((5,5))}')


##Passenger P2 drop
state_space[L2]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,L2,L1)
V = np.array(V)
print(f'\n Grid for TaxiLocation to P2 drop at L1: \n{V.reshape((5,5))}')

print(f'\n For  5th episode\n')

## Passenger P1 pickup
state_space[L1]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,TaxiLocation,L1)
V = np.array(V)
print(f'\n Grid for TaxiLocation to L1 : \n {V.reshape((5,5))}')

## Passenger P1 Drop
state_space[L2]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,L1,L2)
V = np.array(V)
print(f'\n Grid for TaxiLocation to L4 P1 drop: \n {V.reshape((5,5))}')

##Passenger P2 pickup
state_space[L1]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,L1,L2)
V = np.array(V)
print(f'\n Grid for TaxiLocation to P2 at L2 : \n {V.reshape((5,5))}')


##Passenger P2 drop
state_space[L2]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,L2,L1)
V = np.array(V)
print(f'\n Grid for TaxiLocation to P2 drop at L1: \n{V.reshape((5,5))}')


print(f'\n For  6th episode\n')

## Passenger P1 pickup
state_space[L1]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,TaxiLocation,L1)
V = np.array(V)
print(f'\n Grid for TaxiLocation to L1 : \n {V.reshape((5,5))}')

## Passenger P1 Drop
state_space[L2]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,L1,L2)
V = np.array(V)
print(f'\n Grid for TaxiLocation to L4 P1 drop: \n {V.reshape((5,5))}')

##Passenger P2 pickup
state_space[L1]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,L1,L2)
V = np.array(V)
print(f'\n Grid for TaxiLocation to P2 at L2 : \n {V.reshape((5,5))}')


##Passenger P2 drop
state_space[L2]['reward'] = 20000
state_space[Obstacle]['reward'] = -10000
V = value_iteration(state_space, Actions_1, Value_State)
path = run_episodes(V,Actions_1,L2,L1)
V = np.array(V)
print(f'\n Grid for TaxiLocation to P2 drop at L1: \n{V.reshape((5,5))}')


Iter: 0  delta: 15999.0

 Grid for TaxiLocation to L1 : 
 [[ 1.5999e+04  1.5999e+04 -1.0000e+00 -1.0000e+00 -1.0000e+00]
 [ 1.5999e+04 -1.0000e+00 -1.0000e+00 -1.0000e+00 -1.0000e+00]
 [-1.0000e+00 -1.0000e+00 -1.0000e+00 -1.0000e+00 -1.0000e+00]
 [-1.0000e+00 -1.0000e+00 -1.0000e+00 -1.0000e+00 -1.0000e+00]
 [-1.0000e+00 -1.0000e+00 -1.0000e+00 -1.0000e+00 -1.0000e+00]]
Iter: 0  delta: 16000.0

 Grid for TaxiLocation to L4 P1 drop: 
 [[ 1.5999e+04  1.5999e+04 -1.0000e+00 -1.0000e+00 -1.0000e+00]
 [ 1.5999e+04 -1.0000e+00 -1.0000e+00 -1.0000e+00  1.5999e+04]
 [-1.0000e+00 -1.0000e+00 -1.0000e+00  1.5999e+04  1.5999e+04]
 [-1.0000e+00 -1.0000e+00 -1.0000e+00 -1.0000e+00  1.5999e+04]
 [-1.0000e+00 -1.0000e+00 -1.0000e+00 -1.0000e+00 -1.0000e+00]]

 Grid for TaxiLocation to P2 at L2 : 
 [[ 1.5999e+04  1.5999e+04 -1.0000e+00 -1.0000e+00 -1.0000e+00]
 [ 1.5999e+04 -1.0000e+00 -1.0000e+00 -1.0000e+00  1.5999e+04]
 [-1.0000e+00 -1.0000e+00 -1.0000e+00  1.5999e+04  1.5999e+04]
 [-1.0000e+00 -1